In [1]:
# !pip install gymnasium renderlab
# !pip install opencv-python
# !pip install pygame

In [2]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [3]:
#visualise maze:
rfpMaze = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
maze1 = ["SFFH", "FHHF", "FHFG", "FFFH", "HFHH"]

desc = rfpMaze
mazeSize = [len(desc),len(desc[0])]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = -1
gift_found = False
for i in range(len(desc)):
    if gift_found:
        break
    for j in range(len(desc[i])):
        giftState += 1
        if desc[i][j] == 'G':
            gift_found = True
            break
            
print(giftState)
print(statePositions)

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

18
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]


In [45]:
def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0
    
def getPosition(state):
    for i in range(len(statePositions)):
        for j in range(len(statePositions[i])):
            if statePositions[i][j] == state:
                return i, j
            
def calcReward(state, nextState):
    y1, x1 = getPosition(state)
    y2, x2 = getPosition(nextState)
    y3, x3 = getPosition(giftState)
    
    currentDist = (((y3 - y1)**2)+((x3 - x1)**2))**0.5
    nextDist = (((y3 - y2)**2)+((x3 - x2)**2))**0.5
    
    changeInDist = currentDist-nextDist
    return changeInDist
#     if changeInDist > 0:
#         return changeInDist/2
#     else:
#         return changeInDist

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 0:
        return [1,2]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def pathFound():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[currentState]) > 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0 and 0 in calcPossibleMoves(currentState):
            newState = currentState - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(currentState):
            newState = currentState + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(currentState):
            newState = currentState + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(currentState):
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return True
        currentState = newState
    return False

def updateTable_qLearning(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*max(qTable_1[nextState])) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ

def updateTable_sarsa(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    nextDirection = nextStep(nextState)
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*qTable_1[nextState][nextDirection]) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ, nextDirection

def updateTable_monteCarlo(steps, totalReward):
    global qTable_1
    global currentState
    minChangeInQ = 0
    didConverge = False
    avgReward = totalReward/len(steps)
    for i in range(len(steps)):
        updated = qTable_1[steps[i][0]][steps[i][1]] + alpha*(avgReward-qTable_1[steps[i][0]][steps[i][1]])
        changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
        if i == 0:
            minChangeInQ = changeInQ
        elif changeInQ < minChangeInQ and changeInQ != 0:
            minChangeInQ = changeInQ
        if changeInQ < convergenceThresh:
            if changeInQ > 0 and pathFound():
                didConverge = True
        qTable_1[steps[i][0]][steps[i][1]] = updated
    timesVisited[currentState] += 1
    return didConverge, minChangeInQ

In [50]:
epsilonValue = 0.3
alpha = 0.6
gamma = 0.7
convergenceThresh = 0.001

In [51]:
#Q LEARNING-----------------------------------------------------------------------

qTable_1 = {}
currentState = 0

maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global currentSteps
    global currentState
    if converged:
        break
        
    direction = nextStep(currentState)
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -1
        else:
            reward = 10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
    
    converged, changeInQ = updateTable_qLearning(direction, nextState, reward)
    

    if terminated or truncated or converged:
        observation, info = env.reset()
        currentState = 0
        if not converged:
            currentEpisode += 1
            resetVisited()

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 27/1000
Time: 46.576 sec
Q-Table:
0: [0, 3.6696042944978684, 0.37012405301938683, 0]
1: [-0.2232507980078372, -0.6, 0.1034087255188349, 0]
2: [-0.19284075771494588, 0, -0.07386337537059635, 0]
3: [0, -0.6, 0, 0]
4: [0, 4.074683756398308, -0.9359999999999999, -0.3855055314579773]
5: [0, 0, 0, 0]
6: [0, 0, 0, 0]
7: [0, 0, 0, 0]
8: [0, -0.9359999999999999, 4.755880094045253, -0.32823370037260663]
9: [-0.48323048113532796, 5.98859305431757, 0.1741615110802449, -0.9359999999999999]
10: [-0.35835600403794804, -0.6, 0.008810211512103594, -0.6]
11: [0, 0.49311264907601676, 0, -0.6]
12: [0, 0, 0, 0]
13: [0, 7.410654037534688, 0, 1.827949307858085]
14: [0, 0, 0, 0]
15: [-0.6, 0, 0, 0]
16: [0, 0, 0, -0.6]
17: [-0.6, 0, 9.999580569599999, 0]
18: [0, 0, 0, 0]
19: [0, 0, 0, 0]
change in Q: 0.00063
46.575 seconds to converge


In [52]:
#SARSA-----------------------------------------------------------------------

qTable_1 = {}
currentState = 0

maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25
startOfEpisode = True
direction = None

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global startOfEpisode
    global currentSteps
    global Direction
    global currentState
    if converged:
        break
        
    if startOfEpisode:
        direction = nextStep(currentState)
        startOfEpisode = False
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -1
        else:
            reward = 10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
    
    converged, changeInQ, nextDirection = updateTable_sarsa(direction, nextState, reward)
    direction = nextDirection

    if terminated or truncated or converged:
        observation, info = env.reset()
        currentState = 0
        if not converged:
            currentEpisode += 1
            startOfEpisode = False
            resetVisited()

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 35/1000
Time: 69.623 sec
Q-Table:
0: [0.3658155902884366, 2.6735773180918163, -0.10381392992044942, 1.5405139983510374]
1: [-0.7780273679403499, -0.9359999999999999, -0.56719064498439, 0]
2: [-0.9143682284389549, -0.9743999999999999, -0.9355428912593289, 0]
3: [-0.7628187255962103, -0.9743999999999999, 0, 0]
4: [0, 3.1993472659412046, -0.9743999999999999, -0.994340132584316]
5: [0, 0, 0, 0]
6: [0, 0, 0, 0]
7: [0, 0, 0, 0]
8: [0, -0.6, 3.985384634686431, -1.3896562198677944]
9: [-0.20572789330552788, 5.54150716523432, 0.12015962693980325, -0.6]
10: [0, 0, -0.14164078649987388, -0.84]
11: [0, 0.49311264907601676, 0, 0]
12: [0, 0, 0, 0]
13: [0, 7.303578609064932, -0.9359999999999999, -0.5355331873363428]
14: [0, 0, 0, 0]
15: [-0.6, 0, 0, 0]
16: [0, 0, 0, -0.6]
17: [-0.6, 0, 9.999580569599999, -0.751328137423857]
18: [0, 0, 0, 0]
19: [0, 0, 0, 0]
change in Q: 0.00063
69.622 seconds to converge


In [53]:
# Monte Carlo -----------------------------------------------------------------------

epsilonValue = 0.3
alpha = 0.6
gamma = 0.7
convergenceThresh = 0.001

currentState = 0
qTable_1 = {}
maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25

episodeSteps = []
totalReward = 0

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global currentSteps
    global currentState
    if converged:
        break
        
    direction = nextStep(currentState)
    episodeSteps.append([currentState, direction])
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -1
        else:
            reward = 10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
            
    totalReward += reward
    currentState = nextState

    if terminated or truncated:
        observation, info = env.reset()
        converged, changeInQ = updateTable_monteCarlo(episodeSteps, totalReward)
        episodeSteps = []
        currentState = 0
        currentEpisode += 1
        resetVisited()

        
    if converged:
        end_time = time.time()
        observation, info = env.reset()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Current Path: " + str(episodeSteps))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 529/1000
Current Path: [[0, 2]]
Time: 578.855 sec
Q-Table:
0: [0, 3.713417989815291, 4.512665604696477, 0]
1: [1.0287547378324782, 4.536095873541259, 2.844120224008705, 0]
2: [1.7200539295104378, 1.9734927370112612, 0.13377512583005063, 0]
3: [0.2181179281931227, -0.06862801462284593, 0, 0]
4: [0, 3.570807365431569, 4.991346211596364, 1.5067997512346354]
5: [0, 0, 0, 0]
6: [0, 0, 0, 0]
7: [0, 0, 0, 0]
8: [0, 3.57204087544088, 1.3294706739586741, 0.2855729586780779]
9: [0.28754401244938843, 1.5319654987472726, 0.28509545133638914, 0.3489283205766609]
10: [0.2850972116718673, 0.324956768511563, 0.3442993992021014, 0.22035275812607807]
11: [0, 0, 0, 0.3442993992021014]
12: [0, 0, 0, 0]
13: [0.23324629049204493, 1.8474769534611593, 0.5579293010304442, 0.4439004368776891]
14: [0, 0, 0, 0]
15: [0, 0, 0, 0]
16: [0, 0, 0, 0.6408707170592368]
17: [0.6408707170592368, 0, 1.510834782178534, 1.2021273146979545]
18: [0, 0, 0, 0]
19: [0, 0, 0, 0]
change in Q: 3.57081


KeyboardInterrupt: 

In [ ]:
env.reset()

In [ ]:
# env.close()